In [38]:
# external dependencies
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import dash_bootstrap_components as dbc
from dash import Dash, Input, Output, html, dcc
import plotly.express as px
import os
import dash

In [39]:
# constants file
from constants import *
from graphs import *


__Configuration Parameter__

In [40]:
# population dataset path
Path_to_population_dataset = 'Dataset\\US_Population.csv'

# accident dataset path
Path_to_accident_dataset = 'Dataset\\US_Accidents_Sampled.csv'

# sampling factor (>=1)
Sampling_Factor = 100
Already_Sampled = True # for efficency the dataset is already sampled

### Population Dataset

In [41]:
# load dataset for US population
df_pop = pd.read_csv(Path_to_population_dataset)
df_pop.dropna()
df_pop.head()

,Year,AL,AK,AZ,AR,CA,CO,CT,DE,FL,...,SD,TN,TX,UT,VT,VA,WA,WV,WI,WY
0,2016,4860545,741522,6941072,2989918,39250017,5540545,3576452,952065,20612439,...,865454,6651194,27862596,3051217,624594,8411808,7288000,1831102,5778708,585501
1,2017,4874747,739786,7044008,3001345,39536653,5607154,3573880,961939,20928863,...,869666,6715984,28304596,3101833,623657,8470020,7423362,1818157,5790186,584910
2,2018,4887681,735139,7158024,3009733,39776830,5691287,3573297,971180,21244317,...,878698,6771631,28704330,3153550,624344,8517685,7535591,1804291,5807406,578668
3,2019,4903185,731158,7278717,3017804,39576757,5758736,3571520,981822,21538187,...,882235,6829174,29145505,3205958,624358,8565256,7614893,1792147,5822434,577601
4,2020,4921532,727890,7421401,3029887,39368078,5845526,3565287,990837,21899341,...,886667,6886834,29618533,3251617,623989,8616207,7693612,1778070,5837466,567025


### Accident Dataset

In [42]:

# sample a subset of the database (for efficency)
#df_acc_name=str(Path_to_accident_dataset + '_Sampled' + str(Sampling_Factor))
#if(not(os.path.isfile(df_acc_name))):
    # import dataset for US accident
#    df_acc = pd.read_csv(Path_to_accident_dataset)
#    number_of_samples = (df_acc.index)/Sampling_Factor
#    df_acc = df_acc.sample(n = number_of_samples, random_state = 351)
#    df_acc.to_csv(Path_to_accident_dataset)
#else:
    # import dataset for US accident
#    df_acc = pd.read_csv(Path_to_accident_dataset)


df_acc = pd.read_csv(Path_to_accident_dataset)

# remove unecessary columns
df_acc.drop(['Source', 'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Description', 'Airport_Code'], axis=1, inplace=True)

# remove null values
df_acc.dropna()



if(not(Already_Sampled)):
    number_of_samples = (df_acc.index)/Sampling_Factor
    df_acc = df_acc.sample(n = number_of_samples, random_state = 351)

# Convert 'Start_Time' and 'End_Time to datetime
df_acc['Start_Time'] = pd.to_datetime(df_acc['Start_Time'],format='ISO8601')
df_acc['End_Time'] = pd.to_datetime(df_acc['End_Time'],format='ISO8601')

df_acc['Year'] = df_acc['Start_Time'].dt.year

# remove year 2023 because it is incomplete
df_acc = df_acc[(df_acc['Start_Time'].dt.year != 2023)]

df_acc.head()
#df_acc.to_csv('US_Accidents_Sampled.csv')

,Unnamed: 0,ID,Severity,Start_Time,End_Time,Distance(mi),Street,City,County,State,...,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Year
0,2909380,A-2919259,3,2018-02-28 19:10:31,2018-02-28 19:55:17,0.00,Carson St,Lakewood,Los Angeles,CA,...,False,False,False,False,False,Night,Night,Night,Day,2018
1,4123847,A-4154878,2,2022-11-22 16:09:03,2022-11-22 17:26:55,0.99,I-40 W,Nashville,Davidson,TN,...,False,False,False,False,False,Day,Day,Day,Day,2022
2,7006131,A-7055452,2,2020-06-24 20:28:00,2020-06-24 21:02:05,0.00,I-94 E,Minneapolis,Hennepin,MN,...,False,False,False,False,False,Day,Day,Day,Day,2020
3,1191012,A-1200791,2,2021-01-08 18:01:03,2021-01-08 18:47:15,0.00,S Choctaw Dr,Baton Rouge,East Baton Rouge,LA,...,False,False,False,True,False,Night,Night,Day,Day,2021
4,453540,A-453553,2,2017-05-11 08:05:29,2017-05-11 08:34:56,0.02,Meridian Ave N,Seattle,King,WA,...,False,False,False,True,False,Day,Day,Day,Day,2017


__Dashboard Layout__

```
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([

    # title row
    dbc.Row(dbc.Col(html.H1("USA Accidents"), width="auto"), justify="center",), 
    html.Br(),

    # top graphs row
    dbc.Row([

        # top-left graph col
        dbc.Col([
            # top-left graph options
            dbc.Row([
                # time interval select
                dbc.Col([
                    dbc.Row(
                        dbc.Col(
                            html.P("Time Interval: "), width="auto"
                        ), 
                        justify="center"
                    ),
                    dbc.Row(
                        dbc.Select(
                            options=[
                                {"label":"Monthly", "value":"Monthly"}, 
                                {"label":"Yearly", "value":"Yearly"}, 
                                {"label":"Daily", "value":"Daily"},
                                {"label":"Hourly", "value":"Hourly"}  
                            ],
                            value='Monthly',
                            id="Time-Interval-Select"
                        )
                    )
                ]),
                # view mode select
                dbc.Col([
                    dbc.Row(
                        dbc.Col(
                            html.P("View Mode: "), width="auto"
                        ), 
                        justify="center"
                    ),
                    dbc.Row(
                        dbc.RadioItems(
                            options=[
                                {"label":"Grouped", "value":"Grouped"}, 
                                {"label":"Separated", "value":"Separated"}
                            ],
                            value = 'Grouped',
                            id='ViewMode-Select'
                        )
                    )
                ])
            ]),
            # top-left graph
            dbc.Row([
                 # top-left graph
                dbc.Col([
                    # barchart
                    dcc.Graph(id='Temporal-Distribution-acc-graph')
                ])
            ])
        ]),

        # top-right graph col
        dbc.Col([
            # top-right graph option
            dbc.Row([
                dbc.Row(
                    dbc.Col(
                            html.P("Period: "), width="auto"
                        ), 
                        justify="center"
                    ),
                dbc.Select(
                    options=[
                        {"label":"All", "value":"all"}, 
                        {"label":"2016", "value":"2016"}, 
                        {"label":"2017", "value":"2017"},
                        {"label":"2018", "value":"2018"},
                        {"label":"2019", "value":"2019"},
                        {"label":"2020", "value":"2020"},
                        {"label":"2021", "value":"2021"},
                        {"label":"2022", "value":"2022"}        
                    ],
                    value='all',
                    id="Time-Interval-Select-pie"
                )
            ]),
            dbc.Row([
                dcc.Graph(id='Severity-distribution-pie')
            ])
        ]),
    ]), 
    # bottom graphs row
    dbc.Row([
        # bottom-left graph col
        dbc.Col([
            # bottom-left graph options
            dbc.Row([
                # period select
                dbc.Col([
                    dbc.Row(
                        dbc.Col(
                            html.P("Period: "), width="auto"
                        ), 
                        justify="center"
                    ),
                    # bottom-left graph option
                    dbc.Row([
                        dbc.Select(
                            options=[
                                {"label":"2016", "value":"2016"}, 
                                {"label":"2017", "value":"2017"},
                                {"label":"2018", "value":"2018"},
                                {"label":"2019", "value":"2019"},
                                {"label":"2020", "value":"2020"},
                                {"label":"2021", "value":"2021"},
                                {"label":"2022", "value":"2022"}        
                            ],
                            value='2022',
                            id="Accident_per_populetion_period_select"
                        ),
                        dbc.Select(
                            options=[
                                {"label":"None", "value":"None"}, 
                                {"label":"Worst to best", "value":"WorstToBest"},
                                {"label":"Best to worst", "value":"BestToWorst"},      
                            ],
                            value='None',
                            id="Accident_per_populetion_order_select"
                        ),

                    ]),
                    dbc.Row([
                        dcc.Graph(id='Accident_per_populetion_barchart')
                    ])
                ])
            ])
        ]),
    ])
])


    

app.run(debug=True)
```

In [43]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([

    # title row
    dbc.Row(dbc.Col(html.H1("USA Accidents"), width="auto"), justify="center", className="mb-2"), 

    # top graphs row
    dbc.Row([

        # top-left graph col
        dbc.Col([
            # top-left graph options
            dbc.Row([
                # time interval select
                dbc.Col([
                    html.P("Time Interval:"),
                    dbc.Select(
                        options=[
                            {"label":"Monthly", "value":"Monthly"}, 
                            {"label":"Yearly", "value":"Yearly"}, 
                            {"label":"Daily", "value":"Daily"},
                            {"label":"Hourly", "value":"Hourly"}  
                        ],
                        value='Monthly',
                        id="Time-Interval-Select"
                    )
                ], width=6),
                # view mode select
                dbc.Col([
                    html.P("View Mode:"),
                    dbc.RadioItems(
                        options=[
                            {"label":"Grouped", "value":"Grouped"}, 
                            {"label":"Separated", "value":"Separated"}
                        ],
                        value='Grouped',
                        id='ViewMode-Select'
                    )
                ], width=6)
            ], className="mb-2"),
            # top-left graph
            dcc.Graph(id='Temporal-Distribution-acc-graph')
        ], width=6),

        # top-right graph col
        dbc.Col([
            # top-right graph option
            html.P("Period:"),
            dbc.Select(
                options=[
                    {"label":"All", "value":"all"}, 
                    {"label":"2016", "value":"2016"}, 
                    {"label":"2017", "value":"2017"},
                    {"label":"2018", "value":"2018"},
                    {"label":"2019", "value":"2019"},
                    {"label":"2020", "value":"2020"},
                    {"label":"2021", "value":"2021"},
                    {"label":"2022", "value":"2022"}        
                ],
                value='all',
                id="Time-Interval-Select-pie"
            ),
            dcc.Graph(id='Severity-distribution-pie')
        ], width=6),
    ], className="mb-2"), 

    # bottom graphs row
    dbc.Row([
        # bottom-left graph col
        dbc.Col([
            # bottom-left graph options
            html.P("Period:"),
            dbc.Select(
                options=[
                    {"label":"2016", "value":"2016"}, 
                    {"label":"2017", "value":"2017"},
                    {"label":"2018", "value":"2018"},
                    {"label":"2019", "value":"2019"},
                    {"label":"2020", "value":"2020"},
                    {"label":"2021", "value":"2021"},
                    {"label":"2022", "value":"2022"}        
                ],
                value='2022',
                id="Accident_per_populetion_period_select"
            ),
            html.P("Order:"),
            dbc.Select(
                options=[
                    {"label":"None", "value":"None"}, 
                    {"label":"Worst to best", "value":"WorstToBest"},
                    {"label":"Best to worst", "value":"BestToWorst"},      
                ],
                value='None',
                id="Accident_per_populetion_order_select"
            ),
            dcc.Graph(id='Accident_per_populetion_barchart')
        ], width=12),
    ])
], fluid=True, className="p-3")
    

#app.run(debug=True)

In [44]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# input components:

# first graph options
input1 = dbc.Col(
    dbc.Row(
        [
        dbc.Col([
            html.P("Time Interval:"),
            dbc.Select(
                options=[
                        {"label":"Monthly", "value":"Monthly"}, 
                        {"label":"Yearly", "value":"Yearly"}, 
                        {"label":"Daily", "value":"Daily"},
                        {"label":"Hourly", "value":"Hourly"}  
                ],
                value='Monthly',
                id="Time-Interval-Select"
            )
        ], width = 6
        ),
        dbc.Col([
                html.P("View Mode:"),
                dbc.RadioItems(
                    options=[
                            {"label":"Grouped", "value":"Grouped"}, 
                            {"label":"Separated", "value":"Separated"}
                    ],
                    value='Grouped',
                    id='ViewMode-Select'
                )
            ], width = 6)
        ]
    )
)
# secong graph options
input2= dbc.Col(
    [
        html.P("Period:"),
        dbc.Select(
            options=[
                    {"label":"All", "value":"all"}, 
                    {"label":"2016", "value":"2016"}, 
                    {"label":"2017", "value":"2017"},
                    {"label":"2018", "value":"2018"},
                    {"label":"2019", "value":"2019"},
                    {"label":"2020", "value":"2020"},
                    {"label":"2021", "value":"2021"},
                    {"label":"2022", "value":"2022"}        
            ],
            value='all',
            id="Time-Interval-Select-pie"
        ),
        
    ]
)



# graphs:
FirstGraph = dbc.Col(
    [dcc.Graph(id='Temporal-Distribution-acc-graph')]
)

SecondGraph = dbc.Col(
    [dcc.Graph(id='Severity-distribution-pie')]
)

# page components:

# title
title = dbc.Row(
    dbc.Col(html.H1("USA Accidents"), width="auto"),
    justify="center",
    className="mb-2",
    style = {'backgroundColor': 'red'}
)

FirstInputRow = dbc.Row(
    [input1, input2]
)

FirstRow = dbc.Row(
    [FirstGraph, SecondGraph]
)

SecondRow = dbc.Row(
    []
)

lcol =  dbc.Col(
         [FirstInputRow, FirstRow, SecondRow],
         style = {'backgroundColor': 'yellow'},
         width=8
)

# right column
rcol = dbc.Col(
         html.Div("Contenuto della Colonna 4"),
         style = {'backgroundColor': 'orange'},
         width=4
)




# app layout
app.layout=dbc.Container([
    dbc.Col([
        title,
        dbc.Row([lcol, rcol])
        ]
    )
],
style={'width' : '100%'},
fluid = True,
)

app.run(debug=True)

__Top-left graph callback__

In [45]:
@app.callback(Output("Temporal-Distribution-acc-graph", "figure"),[Input("Time-Interval-Select", "value"), Input("ViewMode-Select", "value")])
def TDABarchart(time_interval, ViewMode):
    if(ViewMode == 'Grouped'):
        fig = SingleBarChart(df_acc, time_interval, Sampling_Factor)
        return fig
    else:
        if(ViewMode == 'Separated'):
            fig = MultiBarChart(df_acc, time_interval, Sampling_Factor)
            return fig
        else:
            return 0

__Top-right graph callback__

In [46]:
@app.callback(Output("Severity-distribution-pie", "figure"),[Input("Time-Interval-Select-pie", "value")])
def SDPiechart(time_interval):
    fig = PieChart(df_acc, time_interval)
    return fig

__Bottom-Left graph callback__

In [47]:
@app.callback(Output("Accident_per_populetion_barchart", "figure"),[Input("Accident_per_populetion_period_select", "value"),Input("Accident_per_populetion_order_select", "value")])
def WorstBestPieChart(time_interval, order_by):
    fig = BestWorstAcc(df_acc, df_pop, time_interval, order_by, Sampling_Factor)
    return fig

http://127.0.0.1:8050